# Video-RAG using Ragie

This notebook demonstrates how to build a video-based Retrieval Augmented Generation (RAG) system. It uses [Ragie's](https://www.ragie.ai/) video ingestion and retrieval capabilities to enable semantic search and Q&A over video content.

In [2]:
import os
import time

from dotenv import load_dotenv
from ragie import Ragie

load_dotenv()

True

In [3]:
ragie = Ragie(
    auth=os.getenv('RAGIE_API_KEY')
)

## Remove previous data

In [4]:
response = ragie.documents.list()       
documents = response.result.documents

for document in documents:
    try:
        ragie.documents.delete(
            document_id=document.id
        )
        print(f"Deleted document {document.id}")
    except Exception as e:
        print(f"Error deleting document {document.id}: {e}")

Deleted document 58d43630-ae5b-4ef5-b361-93e7f97c5b67


## Ingestion

In [5]:
current_dir = os.path.curdir
file_path = os.path.join(current_dir, 'videos/messi-goals.mp4')
result = ragie.documents.create(request={
    "file": {
        "file_name": "messi-goals.mp4",
        "content": open(file_path, "rb"),
    },
    "mode": {
        "video": "audio_video"
    }
})

print(result)

status='partitioning' id='a24c92c4-0856-4c8b-8054-d34133b6c050' created_at=datetime.datetime(2025, 6, 17, 18, 35, 51, 854005, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 6, 17, 18, 35, 52, 233392, tzinfo=TzInfo(UTC)) name='messi-goals.mp4' metadata={} partition='default' chunk_count=None external_id=None page_count=None


## Retrieval

In [12]:
response =  ragie.retrievals.retrieve(request={
    "query": "Give detailed description of the video with timestamps of the events"
})

print(response)

scored_chunks=[ScoredChunk(text='{"video_description": "Okay, here\'s a detailed description of the video you sent:\\n\\n**General Description**\\n\\nThe video captures a goal scored by Lionel Messi during a FIFA World Cup match between Argentina and Mexico.  The angle of the video is from behind the goal at a high angle, allowing a clear view of the pitch, the players, and the crowd. The stadium is packed with spectators, and the overall atmosphere seems highly charged.\\n\\n**Visual Elements**\\n\\n*   **Scene:** The video shows a wide shot of the football pitch. We see the green grass of the field, various players in their respective kits (Argentina in light blue and white stripes, Mexico in green and white), and the referee in black.\\n*   **Details:** Advertising boards line the edge of the pitch, featuring logos like \\"Qatar Airways\\" and \\"Qatar Energy.\\" The stadium is filled with a large crowd, mostly out of focus.\\n*   **Action:**\\n    *   The game appears to be mid-act

In [13]:
content = []
for chunk in response.scored_chunks:
    print(chunk.text)
    print("#"*100)
    content.append({
            **chunk.document_metadata,
            "text": chunk.text,
            "document_name": chunk.document_name,
            "start_time": chunk.metadata.get("start_time"),
            "end_time": chunk.metadata.get("end_time")
        })

{"video_description": "Okay, here's a detailed description of the video you sent:\n\n**General Description**\n\nThe video captures a goal scored by Lionel Messi during a FIFA World Cup match between Argentina and Mexico.  The angle of the video is from behind the goal at a high angle, allowing a clear view of the pitch, the players, and the crowd. The stadium is packed with spectators, and the overall atmosphere seems highly charged.\n\n**Visual Elements**\n\n*   **Scene:** The video shows a wide shot of the football pitch. We see the green grass of the field, various players in their respective kits (Argentina in light blue and white stripes, Mexico in green and white), and the referee in black.\n*   **Details:** Advertising boards line the edge of the pitch, featuring logos like \"Qatar Airways\" and \"Qatar Energy.\" The stadium is filled with a large crowd, mostly out of focus.\n*   **Action:**\n    *   The game appears to be mid-action. We see the ball in play, being passed toward

## Video chunk creation

In [14]:
content[0]

{'text': '{"video_description": "Okay, here\'s a detailed description of the video you sent:\\n\\n**General Description**\\n\\nThe video captures a goal scored by Lionel Messi during a FIFA World Cup match between Argentina and Mexico.  The angle of the video is from behind the goal at a high angle, allowing a clear view of the pitch, the players, and the crowd. The stadium is packed with spectators, and the overall atmosphere seems highly charged.\\n\\n**Visual Elements**\\n\\n*   **Scene:** The video shows a wide shot of the football pitch. We see the green grass of the field, various players in their respective kits (Argentina in light blue and white stripes, Mexico in green and white), and the referee in black.\\n*   **Details:** Advertising boards line the edge of the pitch, featuring logos like \\"Qatar Airways\\" and \\"Qatar Energy.\\" The stadium is filled with a large crowd, mostly out of focus.\\n*   **Action:**\\n    *   The game appears to be mid-action. We see the ball in

Let's see the above video clip to verify the video_description is correct

In [16]:
from moviepy import VideoFileClip

video = VideoFileClip("videos/messi-goals.mp4")
goal_clip = video.subclipped(content[0]['start_time'], content[0]['end_time'])
goal_clip.write_videofile("videos/goal_clip.mp4")

MoviePy - Building video videos/goal_clip.mp4.
MoviePy - Writing audio in goal_clipTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video videos/goal_clip.mp4



MoviePy - Done !
MoviePy - video ready videos/goal_clip.mp4


In [17]:
from IPython.display import HTML
from pathlib import Path

# View the created video chunk
video_path = Path('videos/goal_clip.mp4')
if video_path.exists():
    video_html = f'''
    <video width="640" height="480" controls>
        <source src="{video_path}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    '''
    display(HTML(video_html))
else:
    print("Video file not found!")